- https://www.youtube.com/watch?v=9sHcLvVXsns&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=9

# 1 torchtext Field,  TabularDataset, BucketIterator (toy example)

- step 
    - 1. specify how preprocessing should be done -> Fields
    - 2. use dataset to load the data -> Tabular (JSON,csv,tsv)
    - 3. Construct an iterator to do batch, padding -> BucketIterator

In [2]:
from  torchtext.data import Field, TabularDataset, BucketIterator

class `torchtext.data.Field(sequential=True, use_vocab=True, init_token=None, eos_token=None, fix_length=None, dtype=torch.int64, preprocessing=None, postprocessing=None, lower=False, tokenize=None, tokenizer_language='en', include_lengths=False, batch_first=False, pad_token='<pad>', unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False)`

In [5]:
# 여기서는 간단한 토크나이저 활용
tokenizer = lambda x : x.split()

# 어떤 식으로 전처리할지 
quote = Field(sequential=True, use_vocab=True, tokenize=tokenizer, lower=True)
score = Field(sequential=False, use_vocab=False)

# score 는 감정분석에서 긍부정 레이블, 질의응답에서 정답 등에 해당
# csv는 '컬럼이름' : (원하는 이름, 컬럼이름) 의 형태
fields = {'quote' : ('q', quote), 'score' : ('s', score)}

# 훈련, 테스트로 나누기
trainData, testData = TabularDataset.splits(
                                            path='',
                                            train = 'train.json',
                                            test='test.json',
                                            # validation='val'
                                            format='json',
                                            fields=fields)

`TabularDataset` fields

- tuple(str, Field)]: If using a list, the format must be CSV or TSV, and the values of the list should be tuples of (name, field). The fields should be in the same order as the columns in the CSV or TSV file, while tuples of (name, None) represent columns that will be ignored.

- If using a dict, the keys should be a subset of the JSON keys or CSV/TSV columns, and the values should be tuples of (name, field). Keys not present in the input dictionary are ignored. This allows the user to rename columns from their JSON/CSV/TSV key names and also enables selecting a subset of columns to load.

In [12]:
trainData[0]

In [18]:
trainData[0].__dict__.keys()

dict_keys(['q', 's'])

In [19]:
trainData[0].__dict__.values()

dict_values([['you', 'must', 'own', 'everything', 'in', 'your', 'world.', 'there', 'is', 'no', 'one', 'else', 'to', 'blame.'], '1'])

In [23]:
trainData[0].__dict__.values()

dict_values([['you', 'must', 'own', 'everything', 'in', 'your', 'world.', 'there', 'is', 'no', 'one', 'else', 'to', 'blame.'], '1'])

In [21]:
# trainData vocab 만들기
# Construct the Vocab object for this field from one or more datasets.
quote.build_vocab(trainData, max_size=10000, min_freq=1)

# batch와 padding 기능
# 모두 같은 길이가 된다
trainIterator, testIterator = BucketIterator.splits(
    (trainData, testData),
    batch_size=2
    # device='cuda'
)

for batch in trainIterator:
    print(batch.q)
    print(batch.s)

tensor([[33, 27],
        [19, 29],
        [24,  7],
        [14, 26],
        [15, 18],
        [34,  2],
        [32, 25],
        [31,  1],
        [16,  1],
        [20,  1],
        [22,  1],
        [12,  1],
        [ 5,  1],
        [ 8,  1]])
tensor([1, 0])
tensor([[10],
        [21],
        [ 4],
        [ 3],
        [ 6],
        [11],
        [17],
        [ 4],
        [ 3],
        [30],
        [28],
        [ 5],
        [13],
        [ 2],
        [ 9],
        [23]])
tensor([1])


padding으로 1이 들어갔다.

## Summary

In [15]:
tokenizer = lambda x : x.split()

quote = Field(sequential=True, use_vocab=True, tokenize=tokenizer, lower=True)
score = Field(sequential=False, use_vocab=False)

fields = {'quote' : ('q', quote), 'score' : ('s', score)}

trainData, testData = TabularDataset.splits(
                                            path='',
                                            train = 'train.csv',
                                            test='test.csv',
                                            # validation='val'
                                            format='csv',
                                            fields=fields)

quote.build_vocab(trainData, max_size=10000, min_freq=1)

device = 'cpu'

trainIterator, testIterator = BucketIterator.splits(
    (trainData, testData),
    batch_size=2,
    device=device)

for batch in trainIterator:
    print(batch.q)
    print(batch.s)

tensor([[33, 27],
        [19, 29],
        [24,  7],
        [14, 26],
        [15, 18],
        [34,  2],
        [32, 25],
        [31,  1],
        [16,  1],
        [20,  1],
        [22,  1],
        [12,  1],
        [ 5,  1],
        [ 8,  1]])
tensor([1, 0])
tensor([[10],
        [21],
        [ 4],
        [ 3],
        [ 6],
        [11],
        [17],
        [ 4],
        [ 3],
        [30],
        [28],
        [ 5],
        [13],
        [ 2],
        [ 9],
        [23]])
tensor([1])


# 2 torchtext.datasets

`torchtext.datasets` 이용 -> Machine translation

In [3]:
from  torchtext.data import Field, BucketIterator
import spacy
from torchtext.datasets import Multi30k # 데이터셋

In [24]:
# 토크나이저
spacyEng = spacy.load('en')
spacyGer = spacy.load('de')

def tokenizeEng(text):
    return [tok.text for tok in spacyEng.tokenizer(text)]
def tokenizeGer(text):
    return [tok.text for tok in spacyGer.tokenizer(text)]

tokenizeEng("hi What's your name")

['hi', 'What', "'s", 'your', 'name']

In [1]:
# 거의 다 동일
english = Field(sequential=True, use_vocab=True, tokenize=tokenizeEng)
german = Field(sequential=True, use_vocab=True, tokenize=tokenizeGer)

trainData, validationData, testData = Multi30k.splits(exts=('.de','.en'),
                                                     fields=(german, english))

english.build_vocab(trainData, max_size=10000, min_freq=2)
german.build_vocab(trainData, max_size=10000, min_freq=2)

trainIterator, validationIterator, testIterator = BucketIterator.splits(
    (trainData, validationData, testData),
    batch_size=32,
    device = 'cpu')

for batch in trainIterator:
    print(batch)
    break

In [19]:
english.vocab.stoi['me']

5627

In [23]:
english.vocab.itos[7]

'on'

# 3 한국어 example
- https://wikidocs.net/65348 

In [1]:
import urllib.request
import pandas as pd

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f34276d6490>)

In [3]:
trainDf = pd.read_table('ratings_train.txt')
testDf = pd.read_table('ratings_test.txt')

In [4]:
trainDf.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 필드 정의

In [5]:
from torchtext.data import Field
from konlpy.tag import Mecab

In [55]:
tokenizer = Mecab()

text = Field(sequential=True, use_vocab=True, 
             tokenize=tokenizer.morphs,
             batch_first=True, fix_length=20)

label = Field(sequential=False, use_vocab=False, is_target=True)

fields = {'document' : ('doc', text), 'label' : ('y', label)}

## 데이터셋 만들기

In [56]:
from torchtext.data import TabularDataset

In [57]:
trainData, testData = TabularDataset.splits(
    path='', train='ratings_train.txt', test='ratings_test.txt',
    format='tsv', fields=fields)

In [58]:
vars(trainData[0])

{'doc': ['아', '더', '빙', '.', '.', '진짜', '짜증', '나', '네요', '목소리'], 'y': '0'}

## 단어 집합 만들기

In [59]:
text.build_vocab(trainData, min_freq=10, max_size=10000)

In [60]:
print(len(text.vocab))

10002


## 데이터로더 만들기

In [47]:
from torchtext.data import Iterator, BucketIterator

In [64]:
batchSize=5

trainLoader = Iterator(trainData, batch_size=batchSize)
testLoader = Iterator(testData, batch_size=batchSize)

In [65]:
for batch in trainLoader:
    print(batch)
    break


[torchtext.data.batch.Batch of size 5]
	[.doc]:[torch.LongTensor of size 5x20]
	[.y]:[torch.LongTensor of size 5]


In [54]:
print(batch.doc.shape)
print(batch.doc)

torch.Size([5, 20])
tensor([[   0, 1249,  165,   12,   31,  282,  510,  439,  362,    7,    0,    0,
            0,    0,   63,   12,  661,    7, 7162, 1961],
        [   0, 7041,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [ 207,    0, 6317,    0,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [   0,    0,    0,   19, 9940,    0, 4097,   67,    0,  717,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1],
        [1432,  111, 5038,   25,    0,  219,   47,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])


In [67]:
trainIterator, testIterator = BucketIterator.splits(
    (trainData, testData),
    batch_size=batchSize,
    device = 'cpu')

for batch in trainIterator:
    print(batch)
    break


[torchtext.data.batch.Batch of size 5]
	[.doc]:[torch.LongTensor of size 5x20]
	[.y]:[torch.LongTensor of size 5]


`Iterator, BucketIterator` 둘이 뭐가 다른건지 모르겠다
    - 아마 후자는 (train, val, test) 를 한번에 다룰 수 있는 것 같다.